# CHSH Game
In this code we simulate the thought experiment known as the CHSH (Clauser, Horne, Shimony, Holt) game

We have two parties: "Alice" and "Bob". Each one receives, respectively, bits x and y as inputs, then each one responds with bits a and b. They win the game if a and b are such that the following is satisfied
$$a \oplus b = x.y$$
They can talk and decide a strategy beforehand, but they are unable to comunicate during the game. Classicaly, the best strategy guarantees that they will win with probability $0.75$. But if instead of outputing classical bits they output the measurements of the shared Bell state
$$\frac{|00\rangle + |11\rangle}{\sqrt{2}}$$
it can be shown that the best strategy guarantees a win rate of 
$$\cos^{2}(\pi/8) \approx 0.85$$
It is possible to use this experiment to chech whether the device we are running the code is actually quantum.

The strategy is as follows:

- If Alice receives $x = 0$ she measures her qubit and outputs the result, else she first applies $R_Y(\pi/2)$.
- If Bob receives $y = 0$ he applies $R_Y(\pi/4)$ before measuring, else he applies $R_Y(-\pi/4)$.

In [105]:
%matplotlib inline

from qiskit import *
from qiskit.tools.monitor import *
from qiskit.visualization import *
from qiskit.providers.ibmq import least_busy
import numpy as np

provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-01-06 21:54:56,158: Credentials are already in use. The existing account in the session will be replaced.


In [106]:
# Let us write a function to create the circuit representing the CHSH game
def CHSH_circ(x, y, a0 = 0, a1 = np.pi/2, b0 = np.pi/4, b1 = -np.pi/4):
    
    # x: bit received by Alice
    # y: bit received by Bob
    # a0: Y-Rotation if Alice received a 0
    # a1: Y-Rotation if Alice received a 1
    # b0: Y-Rotation if Bob received a 0
    # b1: Y-Rotation if Bob received a 1
    
    circ = QuantumCircuit(2, 2)
    
    # First we set the Bell state
    circ.h(0)
    circ.cx(0, 1)
    
    # Alice's qubit 
    if x == 0:
        circ.ry(a0, 0)
    else:
        circ.ry(a1, 0)
        
    # Bob's qubit
    if y == 0:
        circ.ry(b0, 1)
    else:
        circ.ry(b1, 1)
    
    circ.measure(range(2), range(2))
     
    return circ
    

In [107]:
# Now we need a function that will calculate the probability of winning the game
def win_prob(backend, shots = 1024):
    
    victories = 0
    
    games = [CHSH_circ(0, 0), CHSH_circ(1, 0), CHSH_circ(0, 1), CHSH_circ(1, 1)]
    job = execute(games, backend = backend, shots = shots)
    job_monitor(job)
    
    # We have four possible games
    # In the first three, they win if they output equal results
    for game in games[0:3]:
        counts = job.result().get_counts(game)
        if '00' in counts:
            victories += counts['00']
        if '11' in counts:
            victories += counts['11']
    # In the fourth, they win if they output different results
    counts = job.result().get_counts(games[3])
    if '01' in counts:
        victories += counts['01']
    if '10' in counts:
        victories += counts['10']
    
    prob = victories/(4*shots)
    
    return prob

In [108]:
# First let us run the games in a simulator
backend = Aer.get_backend('qasm_simulator')
win_prob(backend)

Job Status: job has successfully run


0.85302734375

In [109]:
# Now we run it in a real quantum device
backend = least_busy(provider.backends(operational = True, simulator = False, status_msg = 'active',
                            filters = lambda x: x.configuration().n_qubits >= 2))
print('Executing on:', backend)
win_prob(backend)

Executing on: ibmqx2
Job Status: job has successfully run


0.78466796875